# Data Analytics
## Course Assignment N. 18: Ivy League

Zhe Huang, 2020.5

---
### The goal:
> - Explore and describe the data (i.e., standard descriptive statistics, visualize the variables with different graphs, draw distributions and histograms of variables, are there outliers? Any interesting observation? Any correlations? Etc.)
> - Pre-process the data (i.e., handle and fill unknowns if there are any, etc.)
> - Use at least two different clustering algorithms and compare them against one another. What is the most optimal number of clusters?
> - Evaluate and compare the accuracy of the different models
---

This notebook explores and analyzes the university rankings.


In order to illustrate the interactive graph visualization, Jupyter Notebook provides a tool to load and run the JavaScript. It will fetch the ipynb file from Github.

For this assignment, the link is [here]().


In this project I used `Python 3.7` as programming language. I also used `pyecharts` to draw the interactive pictures and `sklearn` to cluster the data.

In [60]:
import sklearn
import csv
import random
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans, SpectralClustering, AgglomerativeClustering
import sklearn.mixture
import sklearn.metrics as sm
import prettytable

import pyecharts.options as opts
from pyecharts.charts import Scatter3D,Map,Radar,Line

#### 1. Simple preparations.

- `randomcolor()`: generates a random type color.

- `filling_unknowns()`: it will find the **nan** inside the data and will try to set it to average number according to the data from other years. It we cannot find relative data, it will be changed to the average number of the whole column.

- `read_tell()`: reads the given files and generates the dicts based on the input. It reads three ranking data `cwurData`, `shanghaiData` and `timesData`. Then it describes the info briefly.

In [46]:
def randomcolor():
    colorArr = ['1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F']
    color = ""
    for i in range(6):
        color += colorArr[random.randint(0, 14)]
    return "#" + color

def filling_unknowns(data, uni_name, col_name):
    missing_code = pd.isna(data[col_name])
    data.loc[missing_code, col_name] = data[missing_code].apply(
        lambda x: (data[data[uni_name] == x[uni_name]])[col_name].mean(), axis=1)

    missing_code = pd.isna(data[col_name])
    data.loc[missing_code, col_name] = data[col_name].mean()


def read_tell():
    cwur = pd.read_csv('cwurData.csv')
    shanghai = pd.read_csv('shanghaiData.csv')
    times = pd.read_csv('timesData.csv')

    data_dict = {'cwur': cwur, 'shanghai': shanghai, 'times': times}

    for name, data in data_dict.items():
        print('Data source: {} ({} rows, {} columns)'.format(name, len(data), int(data.size / len(data))))

    return cwur, shanghai, times


cwurData, shanghaiData, timesData = read_tell()

Data source: cwur (2200 rows, 14 columns)
Data source: shanghai (4897 rows, 11 columns)
Data source: times (2603 rows, 14 columns)


---
### Question 1
Pre-process the data (i.e., handle and fill unknowns if there are any, etc.)

- CWUR Ranking

Data from CWUR only misses some values from column `broad_impact`. So that, using function `filling_unknowns` is enough. 

In [4]:
def CWUR_pre(cd):
    # nan_data = cwurData[pd.isna(cwurData['broad_impact'])]
    filling_unknowns(cd, 'institution', 'broad_impact')
    return cd

cwurData = CWUR_pre(cwurData)

- Shanghai Ranking

At first it will only keep the rows that containing more than 6 meaningful value. 

Then it checks the column `ns` and set the default value to 0. Then I will find that the data does not include the total score for most of the universites, so that we will calculate it by the given formula from the [official website](http://www.shanghairanking.com/ARWU-Methodology-2015.html).

Then because some world rank are represented as an interval, like [30-50], I rank the university based on the total score calculated above.

![title](shanghai.png)

In [5]:
def Shanghai_pre(sd):
    sd = sd.dropna(thresh=6)

    missing = pd.isna(sd['ns'])
    sd.loc[missing, 'ns'] = 0

    missing = pd.isna(sd['total_score'])
    sd.loc[missing, 'total_score'] = sd[missing].apply(
        lambda x: (0.2 * (x['award'] + x['hici'] + x['ns'] + x['pub']) + 0.1 * (x['alumni'] + x['pcp'])), axis=1)

    a = {x: {x: index + 1 for index, x in
             enumerate(sorted(list(sd.loc[sd['year'] == x, 'total_score']), reverse=True))} for x in
         range(2005, 2016)}
    sd.loc[:, 'world_rank'] = sd.apply(lambda x: a[x.year][x.total_score], axis=1)
    
    return sd

shanghaiData = Shanghai_pre(shanghaiData)

- Times Ranking

It is much more complex than the others. First the original data uses **-** to represent the missing value. I changed it to `np.nan`.
    
Then the columns `female_male_ratio` and `international_students` are in the format of string, like `70:30` and `36%`. I changed them into the float numbers, like `0.7` and `0.36`.

Columns `num_students`, `international`, `income` and `total_score` contains a comma, **,** inside the value so that they are extracted as type `string`, I changed them into the float of int values.

Using function `filling_unknowns` to fill the data for most of the columns.

Similar to `Shanghai Ranking`, it doesn't show the total_score for most of the universites and uses interval to represent the ranking, I do the same steps. Also the [offical website](https://www.timeshighereducation.com/news/ranking-methodology-2016).

![](https://www.timeshighereducation.com/sites/default/files/subject-rankings-methodology-2015-large.jpg)

In [62]:
def Times_pre(td):
    td = td.applymap(lambda x: x if x != '-' else np.nan)

    td['female_male_ratio'] = td['female_male_ratio'].apply(
        lambda x: np.nan if (pd.isna(x) or x == '-') else float(x[:2]) / 100.0)
    td.rename(columns={'female_male_ratio': 'female_ratio'}, inplace=True)
    td['international_students'] = td['international_students'].apply(
        lambda x: np.nan if pd.isna(x) else float(x.replace('%', '')) / 100.0)
    td['num_students'] = td['num_students'].apply(lambda x: np.nan if pd.isna(x) else int(x.replace(',', '')))
    td['international'] = td['international'].apply(
        lambda x: np.nan if pd.isna(x) else float(x.replace(',', '')))
    td['income'] = td['income'].apply(lambda x: np.nan if pd.isna(x) else float(x.replace(',', '')))
    td['total_score'] = td['total_score'].apply(lambda x: np.nan if pd.isna(x) else float(x.replace(',', '')))

    filling_unknowns(td, 'university_name', 'female_ratio')
    filling_unknowns(td, 'university_name', 'international_students')
    filling_unknowns(td, 'university_name', 'student_staff_ratio')
    filling_unknowns(td, 'university_name', 'num_students')
    filling_unknowns(td, 'university_name', 'international')
    filling_unknowns(td, 'university_name', 'income')

    missing = pd.isna(td['total_score'])
    td.loc[missing, 'total_score'] = td[missing].apply(lambda x: (
            0.3 * (x['research'] + x['citations'] + x['teaching']) + 0.075 * x['international'] + 0.025 * x['income']),
                                                       axis=1)

    missing = td.world_rank.str.contains('=')
    td.loc[missing, 'world_rank'] = td.world_rank[missing].apply(lambda x: x.replace('=', ''))

    # missing = timesData['world_rank'].str.contains('-')
    a = {x: {x: index + 1 for index, x in
             enumerate(sorted(list(td.loc[td['year'] == x, 'total_score']), reverse=True))} for x in
         range(2011, 2017)}
    td.loc[:, 'world_rank'] = td.apply(lambda x: a[x.year][x.total_score], axis=1)

    return td


timesData = Times_pre(timesData)

In [29]:
print(cwurData)

      world_rank                            institution         country  \
0              1                     Harvard University             USA   
1              2  Massachusetts Institute of Technology             USA   
2              3                    Stanford University             USA   
3              4                University of Cambridge  United Kingdom   
4              5     California Institute of Technology             USA   
...          ...                                    ...             ...   
2195         996              University of the Algarve        Portugal   
2196         997                  Alexandria University           Egypt   
2197         998            Federal University of Ceará          Brazil   
2198         999                 University of A Coruña           Spain   
2199        1000        China Pharmaceutical University           China   

      national_rank  quality_of_education  alumni_employment  \
0                 1                

In [30]:
print (shanghaiData)

      world_rank                              university_name national_rank  \
0              1                           Harvard University             1   
1              2                      University of Cambridge             1   
2              3                          Stanford University             2   
3              4           University of California, Berkeley             3   
4              5  Massachusetts Institute of Technology (MIT)             4   
...          ...                                          ...           ...   
4892         446                        University of Trieste         11-20   
4893         421                       University of Zaragoza          9-13   
4894         405                        Utah State University       126-146   
4895         424              Vienna University of Technology           4-6   
4896         410                       Wake Forest University       126-146   

      total_score  alumni  award   hici     ns    p

In [31]:
print(timesData)

      world_rank                        university_name  \
0              1                     Harvard University   
1              2     California Institute of Technology   
2              3  Massachusetts Institute of Technology   
3              4                    Stanford University   
4              5                   Princeton University   
...          ...                                    ...   
2598         705                    Yeungnam University   
2599         786            Yıldız Technical University   
2600         619               Yokohama City University   
2601         740           Yokohama National University   
2602         657                     Yuan Ze University   

                       country  teaching  international  research  citations  \
0     United States of America      99.7           72.4      98.7       98.8   
1     United States of America      97.7           54.6      98.0       99.9   
2     United States of America      97.8           

---
### Question 2
Explore and describe the data (i.e., standard descriptive statistics, visualize the variables with different graphs, draw distributions and histograms of variables, are there outliers? Any interesting observation? Any correlations? Etc.)

I use `pyecharts` to illustrate the final results. Since it contains more than 3 dimensions, I choose `cwur_score`, `sh_total_score` and `times_total_score` as the x-axis, y-axis and z-axis.

In [34]:
def scatter3d():
    data = [[item.cwur_score,item.sh_total_score,item.times_total_score,1,10,item.university,]
            for index, item in total_pd.iterrows()]

    c = Scatter3D().add(
            series_name="university_ranking",
            data=data,
            xaxis3d_opts=opts.Axis3DOpts(name="cwur_score",type_="value",),
            yaxis3d_opts=opts.Axis3DOpts(name="sh_total_score",type_="value",),
            zaxis3d_opts=opts.Axis3DOpts(name="times_total_score",type_="value",),
            grid3d_opts=opts.Grid3DOpts(width=100, height=100, depth=100),
        )
    return c

c = scatter3d()

c.render_notebook()

Then I show the average score by country according to the `cwur ranking`. It shows that `Romania`,`Puerto Rico` and  `Cyprus` have the lower scores and `Switzerland`, `United States` and `Israel` have the higher scores. Most of the countries are in the interval [44.4,46.8].

In [44]:
def show_map():
    countries = list(set(cwurData.country))

    aa = [[x,cwurData[(cwurData.country == x)].score.mean()] for x in countries]

    aa[countries.index('USA')][0] = 'United States'
    
    aa.sort(key=lambda x: x[1])
    
    print(aa[:3])
    print(aa[-3:])

    return (
    Map()
        .add("", aa, "world")
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Map"), visualmap_opts=opts.VisualMapOpts(is_piecewise=True,min_=42,max_=54),
        )
    )
    
c = show_map()

[['Romania', 44.13333333333333], ['Puerto Rico', 44.175], ['Cyprus', 44.21]]
[['Switzerland', 51.208846153846146], ['United States', 51.83986038394412], ['Israel', 52.654090909090904]]


In [39]:
c.render_notebook()

Here I use Radar graph to illustrate the **TOP 8** university for shanghai Ranking, year 2015. 

It shows that `Havard University` reaches 100 points except for the **pcp** and `CIT` gets 100 at this indicator. 

It can act as a reference to help user find the advantages and disadvantages for each universites.

In [57]:
def show_radar():

    data = [[item.university_name, [item.alumni, item.award, item.hici, item.ns, item.pub, item.pcp]]
        for index, item in shanghaiData[shanghaiData.year == 2015].iterrows()][:8]

    c = (
        Radar()
            .add_schema(
            schema=[
                opts.RadarIndicatorItem(name="alumni"),opts.RadarIndicatorItem(name="award"),
                opts.RadarIndicatorItem(name="hici"),opts.RadarIndicatorItem(name="ns"),
                opts.RadarIndicatorItem(name="pub"),opts.RadarIndicatorItem(name="pcp"),
            ],
            shape="circle",center=["50%", "50%"],radius="80%",
            splitarea_opt=opts.SplitAreaOpts(
                is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
            ),
            textstyle_opts=opts.TextStyleOpts(color="#000"),
        ).set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    )

    for x in data:
        color = randomcolor()
        c.add(series_name=x[0],data=[x[1]],areastyle_opts=opts.AreaStyleOpts(opacity=0.1, color=color),
            linestyle_opts=opts.LineStyleOpts(width=1, color=color),label_opts=opts.LabelOpts(is_show=False))
        
    return c

c = show_radar()

In [58]:
c.render_notebook()

Here it shows the **TOP 10** universities from 2011 to 2016 from `Times Data`. It contains 12 universites and `University of Oxford`, `Massachusetts Institute of Technology`, `Stanford University`, `Harvard University`, `California Institute of Technology`, `Imperial College London`, `University of Cambridge`, `Princeton University` did not drop from the list.

In [90]:
def TOP_10():
    c = (
        Line()
        .add_xaxis(xaxis_data=range(2011,2017))
        .set_global_opts(
            title_opts=opts.TitleOpts(title=""),
            tooltip_opts=opts.TooltipOpts(trigger="axis"),
            toolbox_opts=opts.ToolboxOpts(is_show=False),
            xaxis_opts=opts.AxisOpts(type_="category", boundary_gap=False),
            legend_opts=opts.LegendOpts(type_='scroll',pos_bottom='96%')
        )
    )
    data = [[item.university_name, [item.alumni, item.award, item.hici, item.ns, item.pub, item.pcp]]
            for index, item in shanghaiData[shanghaiData.year == 2015].iterrows()][:8]
    top_uni = list(set(timesData[timesData.world_rank<=10].university_name))
    for item in top_uni:
        uni_data = [x if x <= 10 else np.nan for x in list(timesData[timesData.university_name==item].world_rank) ]

        c.add_yaxis(
            series_name=item,
            y_axis=uni_data,
            markpoint_opts=opts.MarkPointOpts(
                data=[
                    opts.MarkPointItem(type_="min", name="minimum"),
                ]
            ),
        )
    
    return c

c = TOP_10()

In [91]:
c.render_notebook()

---
### Question 3 & 4
Use at least two different clustering algorithms and compare them against one another. What is the most optimal number of clusters?
Evaluate and compare the accuracy of the different models

At first it needs to find useful data columns from the three datasets. I remove some describing type of data and duplicate columns. 

In [12]:
def generate_col_list():
    col_list = [cwurData.columns.values.tolist(), shanghaiData.columns.values.tolist(),
                timesData.columns.values.tolist()]
    col_list[0].remove('institution')
    col_list[0].remove('country')
    col_list[0].remove('year')
    col_list[1].remove('university_name')
    col_list[1].remove('year')
    col_list[1].remove('national_rank')
    col_list[2].remove('university_name')
    col_list[2].remove('country')
    col_list[2].remove('year')
    
    return col_list

print(generate_col_list())

[['world_rank', 'national_rank', 'quality_of_education', 'alumni_employment', 'quality_of_faculty', 'publications', 'influence', 'citations', 'broad_impact', 'patents', 'score'], ['world_rank', 'total_score', 'alumni', 'award', 'hici', 'ns', 'pub', 'pcp'], ['world_rank', 'teaching', 'international', 'research', 'citations', 'income', 'total_score', 'num_students', 'student_staff_ratio', 'international_students', 'female_ratio']]


Then it will find the intersection of the three datasets, making sure that the final dataset does not contain the nan value. There are **362** universites that appear in all the three ranking.

In [8]:
def find_frequent_uni():
    return list(
        set(cwurData['institution']) & set(shanghaiData['university_name']) & set(timesData['university_name']))

len(find_frequent_uni())

362

This function will flatten three datasets and then put them into a single pandas frame. It calculates the average value for every columns in three ranking and uses numbers to represents the countries.

In [13]:
def flatten(universities, col_list):
    total_info = []
    countries = list(set(cwurData.country))
    for uni in universities:
        frames = [cwurData.loc[cwurData['institution'] == uni],
                  shanghaiData.loc[shanghaiData['university_name'] == uni],
                  timesData.loc[timesData['university_name'] == uni]]
        ind = {'university': uni,
               'country': countries.index(cwurData.loc[cwurData['institution'] == uni, 'country'].iloc[0])}
        for cols, fs, name in zip(col_list, frames, ['cwur', 'sh', 'times']):
            for col in cols:
                ind[name + '_' + col] = fs[col].mean()
        total_info.append(ind)

    over_pd = pd.DataFrame(total_info)

    return over_pd

flatten(find_frequent_uni(),generate_col_list())

,university,country,cwur_world_rank,cwur_national_rank,cwur_quality_of_education,cwur_alumni_employment,cwur_quality_of_faculty,cwur_publications,cwur_influence,cwur_citations,...,times_teaching,times_international,times_research,times_citations,times_income,times_total_score,times_num_students,times_student_staff_ratio,times_international_students,times_female_ratio
0,University of Sussex,38,221.500000,18.500000,144.50,237.50,48.500000,363.500000,255.500000,168.500000,...,33.783333,80.216667,36.083333,89.983333,30.400000,54.800000,12001.0,17.4,0.35,0.54
1,Stellenbosch University,33,308.500000,3.000000,361.00,65.00,214.000000,532.500000,498.500000,315.500000,...,27.960000,47.300000,32.100000,45.020000,97.360000,37.505500,21849.0,23.0,0.08,0.53
2,Leiden University,37,103.333333,2.333333,105.00,382.00,85.333333,81.666667,83.333333,89.333333,...,47.350000,55.250000,58.633333,76.850000,56.933333,60.483333,21222.0,17.1,0.10,0.59
3,University of Erlangen-Nuremberg,32,208.500000,13.500000,226.50,516.00,214.000000,173.000000,205.000000,140.000000,...,36.950000,48.650000,22.550000,72.575000,59.725000,44.766875,36146.0,53.9,0.09,0.49
4,University of Nice Sophia Antipolis,8,518.500000,22.000000,259.00,522.50,214.000000,459.000000,262.000000,560.000000,...,21.400000,79.500000,16.100000,53.700000,28.500000,34.035000,27304.0,20.3,0.22,0.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,Ben-Gurion University of the Negev,3,355.500000,5.000000,361.00,522.50,191.500000,278.500000,380.500000,417.000000,...,25.100000,52.900000,21.300000,30.100000,28.700000,27.635000,16695.0,12.6,0.02,0.49
358,University of Idaho,20,646.500000,187.000000,361.00,522.50,214.000000,652.000000,408.000000,458.500000,...,28.850000,29.700000,17.800000,32.400000,28.950000,26.666250,11641.0,21.5,0.06,0.47
359,University of Chicago,20,9.000000,7.000000,11.25,18.25,8.000000,26.250000,19.000000,20.000000,...,85.550000,60.950000,89.766667,98.250000,36.700000,88.383333,14221.0,6.9,0.21,0.42
360,Monash University,47,157.500000,5.500000,277.00,139.50,214.000000,91.000000,141.000000,216.000000,...,43.666667,79.133333,51.616667,63.816667,65.516667,55.183333,50882.0,40.5,0.36,0.56


This function is the core function. It will take the previous pandas dataframe as input and use four different cluster algorithm to cluster the data. I set the parameter `n_clusters` to **5** for all the methods and want to test which is better.

- KMeans: 
    most widely known clustering algorithm. It will minimize the variance within each cluster.

- Spectral Clustering: 
    It is from linear algebra and uses the top eigenvectors from the distance between points.

- Gaussian Mixture Model: 
    It uses probability density function with a mixture of Gaussian probability distributions.

- Agglomerative Clustering: 
    It will merge examples until the desired number of clusters is achieved.

---

And then, I uses three measures to evaluate the final results.

First, since we cannot obtain the real label or category for each university, It is impossible to use the evaluating function that need the parameter `true_label`. Therefore I can only compare some coefficient that can be calculated only by the input data and the `predict_label`.

- silhouette_score: It is a measure of how similar an object is to its own cluster compared to other clusters. If most objects have a high value, then the clustering configuration is appropriate.

- calinski_harabasz_score: The score is defined as ratio between the within-cluster dispersion and the between-cluster dispersion. So that, the higher the value, the better the clustering result.

- davies_bouldin_score: It is defined as the average similarity measure of each cluster with its most similar cluster, where similarity is the ratio of within-cluster distances to between-cluster distances. So that, the lower values indicates better clustering.

*PS: Part of the definitions are from the sklearn tutorials.

In [92]:
def cluster_methods(over_pd):
    kmodel = KMeans(n_clusters=5)
    kmodel.fit(over_pd)
    khat = kmodel.predict(over_pd)

    gmm = sklearn.mixture.GaussianMixture(n_components=5)
    gmm.fit(over_pd)
    ghat = gmm.predict(over_pd)

    smodel = SpectralClustering(n_clusters=5)
    shat = smodel.fit_predict(over_pd)

    amodel = AgglomerativeClustering(n_clusters=5)
    ahat = amodel.fit_predict(over_pd)

    ks = [sm.silhouette_score(over_pd, khat), sm.calinski_harabasz_score(over_pd, khat),
          sm.davies_bouldin_score(over_pd, khat)]
    gs = [sm.silhouette_score(over_pd, ghat), sm.calinski_harabasz_score(over_pd, ghat),
          sm.davies_bouldin_score(over_pd, ghat)]
    ss = [sm.silhouette_score(over_pd, shat), sm.calinski_harabasz_score(over_pd, shat),
          sm.davies_bouldin_score(over_pd, shat)]
    asc = [sm.silhouette_score(over_pd, ahat), sm.calinski_harabasz_score(over_pd, ahat),
           sm.davies_bouldin_score(over_pd, ahat)]

    over_pd['khat'] = khat
    over_pd['ghat'] = ghat
    over_pd['shat'] = shat
    over_pd['ahat'] = ahat

    return ks, gs, ss, asc

Here is the main function of clustering. First it removes the column `university` and then invokes the `cluster_methods`. 

The result shows that `K-Means`, the best one, has the highest values in **silhouette_score** and **calinski_harabasz_score**. Also it has the lowest value in **davies_bouldin_score**. 

The worst one is `Spectral Clustering`, maybe because of the warning it is not suitable for the not fully connected graph.

In [21]:
def cluster():
    # Cluster the data
    universities = find_frequent_uni()

    col_list = generate_col_list()

    over_pd = flatten(universities, col_list)

    uni_name = list(over_pd.university)
    over_pd = over_pd.drop(columns='university')

    ks, gs, ss, asc = cluster_methods(over_pd)

    over_pd['university'] = uni_name

    table = prettytable.PrettyTable(
        ['Method', 'silhouette_score', 'calinski_harabasz_score', 'davies_bouldin_score'])
    table.add_row(['K-Means'] + ks)
    table.add_row(['Gaussian Mixture Model'] + gs)
    table.add_row(['Spectral Clustering'] + ss)
    table.add_row(['Agglomerative Clustering'] + asc)

    print(table)

    return over_pd


total_pd = cluster()

+--------------------------+----------------------+-------------------------+----------------------+
|          Method          |   silhouette_score   | calinski_harabasz_score | davies_bouldin_score |
+--------------------------+----------------------+-------------------------+----------------------+
|         K-Means          |  0.5441109839989704  |    883.4161769654754    | 0.45568932179382216  |
|  Gaussian Mixture Model  | 0.04198903671178271  |    238.87893858501977   |  5.017784982597147   |
|   Spectral Clustering    | -0.04658441841849895 |   0.45810914721460927   |  80.51669251459319   |
| Agglomerative Clustering |  0.5250365965846607  |     750.814512747334    |  0.4613907716947924  |
+--------------------------+----------------------+-------------------------+----------------------+


/Users/neilhuang/PycharmProjects/DA_2/venv/lib/python3.7/site-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


It shows that the most optimal number of clusters is **7**. It has the lowest `davies_bouldin_score` while keeping `silhouette_score` and `calinski_harabasz_score` in a higher level.

In [20]:
def k_means(over_pd):
    table = prettytable.PrettyTable(
        ['k', 'silhouette_score', 'calinski_harabasz_score', 'davies_bouldin_score'])
    res = []
    for k in range(2,10):
        kmodel = KMeans(n_clusters=k)
        kmodel.fit(over_pd)
        khat = kmodel.predict(over_pd)

        table.add_row([k,sm.silhouette_score(over_pd, khat), sm.calinski_harabasz_score(over_pd, khat),
              sm.davies_bouldin_score(over_pd, khat)])

    print(table)
k_means(total_pd)

+---+--------------------+-------------------------+----------------------+
| k |  silhouette_score  | calinski_harabasz_score | davies_bouldin_score |
+---+--------------------+-------------------------+----------------------+
| 2 | 0.6933965278478789 |    375.42765304917634   |  0.6087469984945519  |
| 3 | 0.6311603700035543 |    505.3037405222036    |  0.5495793000746186  |
| 4 | 0.5409265388559772 |    630.9444437508564    |  0.5700608475809951  |
| 5 | 0.5441109377758053 |    883.4161055951661    | 0.45568936888877165  |
| 6 | 0.5538924220769464 |    1110.4839072583227   | 0.45507150251408685  |
| 7 | 0.5558171609911398 |    1380.0878407853486   |  0.3969548819475048  |
| 8 | 0.5315691589888607 |    1603.8082535371373   |  0.4213666664264851  |
| 9 | 0.5420696621283181 |    2084.7898158260286   |  0.4125664800347732  |
+---+--------------------+-------------------------+----------------------+
